In [1]:
# %load assign3.py
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
import csv
import numpy as np
from string import maketrans
import string
trainlist=[]
testlist=[]
testlistACGT=[]
X = []
Y = []
XForTest = []
Yfortest = []
#DNAdict = {'A':1,'C':2,'G':3,'T':4}
with open('train.csv','rb') as traincsvFile:
    lines=csv.reader(traincsvFile)
    for line in lines:
        trainlist.append(list(line))
        
    firstrow = trainlist[0]
    del trainlist[0] #delete first row of train.csv
    
    #print trainlist
    #trainlistrow = [x[1] for x in trainlist]
    translatetable = maketrans('ACGT', '1234')
    #print trainlist
    for eachrow in trainlist:
        eachrow[0] = string.atoi(eachrow[0])
        # print 'eachrow=' + eachrow[1]
        eachrow[1] = eachrow[1].translate(translatetable)
        #print 'new_eachrow=' + eachrow[1]
        Xstring = '-'.join(eachrow[1]).split('-')
        Xstring = map(eval, Xstring)
        #print Xstring
        X.append(Xstring)
        eachrow[2] = string.atoi(eachrow[2])
        Y.append(eachrow[2])
    #print trainlist
    #print Y
    #print X

    

### begin train model

In [ ]:
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation 
    import numpy
    from keras.optimizers import SGD  
    from keras.layers import LSTM
    from keras import regularizers


    # fix random seed for reproducibility
    seed = 7
    numpy.random.seed(seed)

    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=14, init='uniform', activation='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
#     model.add(Dropout(0.5)) 
    model.add(Dense(8, init='uniform', activation='relu'))
#     model.add(Dropout(0.5)) 
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)  

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    Xinput = np.array(X)
    Yinput = np.array(Y)

    # Fit the model
    model.fit(Xinput, Yinput, epochs=2000, batch_size=10, verbose=2)
    # calculate predictions
    # predictions = model.predict(np.array(X))
    # # round predictions
    # rounded = [round(x[0]) for x in predictions]
    # print(rounded)
# 4. 评估模型
    loss, accuracy = model.evaluate(np.array(X), np.array(Y))
    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 5. 数据预测
    probabilities = model.predict(np.array(X))
    predictions111 = [float(round(x)) for x in probabilities]
    accuracy = numpy.mean(predictions111 == Y)
    print("Prediction Accuracy: %.2f%%" % (accuracy*100))


/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, kernel_initializer="uniform", activity_regularizer=<keras.reg..., kernel_regularizer=<keras.reg..., input_dim=14, activation="relu")`
  app.launch_new_instance()
/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/2000
 - 1s - loss: 0.6993 - acc: 0.4720
Epoch 2/2000
 - 0s - loss: 0.6939 - acc: 0.4920
Epoch 3/2000
 - 0s - loss: 0.6937 - acc: 0.4950
Epoch 4/2000
 - 0s - loss: 0.6936 - acc: 0.4700
Epoch 5/2000
 - 0s - loss: 0.6934 - acc: 0.5000
Epoch 6/2000
 - 0s - loss: 0.6933 - acc: 0.4830
Epoch 7/2000
 - 0s - loss: 0.6933 - acc: 0.4880
Epoch 8/2000
 - 0s - loss: 0.6933 - acc: 0.4910
Epoch 9/2000
 - 0s - loss: 0.6932 - acc: 0.4910
Epoch 10/2000
 - 0s - loss: 0.6932 - acc: 0.4920
Epoch 11/2000
 - 1s - loss: 0.6932 - acc: 0.4900
Epoch 12/2000
 - 1s - loss: 0.6932 - acc: 0.5000
Epoch 13/2000
 - 0s - loss: 0.6932 - acc: 0.4870
Epoch 14/2000
 - 0s - loss: 0.6932 - acc: 0.5010
Epoch 15/2000
 - 1s - loss: 0.6932 - acc: 0.4930
Epoch 16/2000
 - 1s - loss: 0.6932 - acc: 0.4840
Epoch 17/2000
 - 1s - loss: 0.6932 - acc: 0.4930
Epoch 18/2000
 - 0s - loss: 0.6932 - acc: 0.4900
Epoch 19/2000
 - 1s - loss: 0.6932 - acc: 0.5030
Epoch 20/2000
 - 1s - loss: 0.6932 - acc: 0.4940
Epoch 21/2000
 - 1s - loss: 0

### begin test predictions

In [3]:
    with open('test.csv', 'rb') as testcsvFile:
        lines = csv.reader(testcsvFile)
        for line in lines:
            testlist.append(list(line))
            testlistACGT.append(list(line))
        firstrow = testlist[0]
        del testlist[0]  # delete first row of train.csv
        del testlistACGT[0]
        
        # trainlistrow = [x[1] for x in trainlist]
        translatetable = maketrans('ACGT', '1234')
#         print testlist
        for eachrow in testlist:
            #eachrow[0] = string.atoi(eachrow[0])
            # print 'eachrow=' + eachrow[1]
            eachrow[1] = eachrow[1].translate(translatetable)
            # print 'new_eachrow=' + eachrow[1]
            Xstringtest = '-'.join(eachrow[1]).split('-')
            Xstringtest = map(eval, Xstringtest)
            # print Xstring
            XForTest.append(Xstringtest)
            # eachrow[2] = string.atoi(eachrow[2])
            # Yfortest.append(eachrow[2])
        #print testlist
        #print XForTest
        # calculate predictions
        predictions = model.predict(np.array(XForTest))
        # round predictions
        rounded = [int(round(x[0])) for x in predictions]
#         print(rounded)

In [4]:
print type(rounded)


<type 'list'>


### output to csv file

In [5]:
idofdna=[x[0] for x in testlistACGT]
# print idofdna #useful！！！
# print testlistACGT
outputlist = np.c_[np.array(idofdna),np.array(rounded)].tolist()
# print outputlist

In [6]:
print type(outputlist)
firstrowtest = ['id','prediction']
# numpy.savetxt('outputtest.csv', outputlist, delimiter = ',',fmt="%f,%f,%f") 
with open("outputtest.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerow(firstrowtest)
    writer.writerows(outputlist)

<type 'list'>


In [7]:
print type(outputlist)
print firstrowtest

<type 'list'>
['id', 'prediction']
